In [ ]:
class MultitaskBERT(nn.Module):
    '''
    This module should use BERT for 3 tasks:

    - Sentiment classification (predict_sentiment)
    - Paraphrase detection (predict_paraphrase)
    - Semantic Textual Similarity (predict_similarity)
    '''
    def __init__(self, config):
        super(MultitaskBERT, self).__init__()
        # You will want to add layers here to perform the downstream tasks.
        # Pretrain mode does not require updating bert paramters.
        #self.num_labels = config.num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased', local_files_only=config.local_files_only)
        for param in self.bert.parameters():
            if config.option == 'pretrain':
                param.requires_grad = False
            elif config.option == 'finetune':
                param.requires_grad = True
        ### TODO
        ## sst - sentiment, para, sts - semantic text similarity    
        self.drop = torch.nn.Dropout(p=0.3)
        self.sst_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_sst)
        self.para_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_para)
        self.sts_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_sts)

        raise NotImplementedError


    def forward(self, input_ids, attention_mask):
        'Takes a batch of sentences and produces embeddings for them.'
        # The final BERT embedding is the hidden state of [CLS] token (the first token)
        # Here, you can start by just returning the embeddings straight from BERT.
        # When thinking of improvements, you can later try modifying this
        # (e.g., by adding other layers).
        ### TODO
        raise NotImplementedError

In [1]:
from datasets import load_multitask_data, SentenceClassificationDataset, SentencePairDataset
from config import PretrainedConfig

In [4]:
sentiment_data, num_labels, paraphrase_data, similarity_data = load_multitask_data(sentiment_filename='data/ids-sst-train.csv',paraphrase_filename='data/quora-train.csv',similarity_filename='data/sts-train.csv')

Loaded 8544 train examples from data/ids-sst-train.csv
Loaded 141498 train examples from data/quora-train.csv
Loaded 6040 train examples from data/sts-train.csv


In [28]:
config = PretrainedConfig(local_files_only=True)

In [30]:
config.get_config_dict('bert-base-uncased')[0]

{'architectures': ['BertForMaskedLM'],
 'attention_probs_dropout_prob': 0.1,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'transformers_version': '4.6.0.dev0',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [7]:
class tempArgs():
    def __init__(self,):
        self.local_files_only=True


In [8]:
tempargs = tempArgs()

In [9]:
sst_train_data = SentenceClassificationDataset(sentiment_data,tempargs)

In [10]:
batched_sst_data = sst_train_data.collate_fn(sst_train_data.dataset)

In [13]:
para_train_data = SentencePairDataset(paraphrase_data,tempargs,isRegression=False)
sts_strain_data = SentencePairDataset(similarity_data,tempargs,isRegression=True)

In [21]:
para_train_data.dataset

[('how can i master myself in geometry ?',
  'how can i master geometry for the cat-14 ?',
  0,
  'a4da64a4e943bf8b008d174ca'),
 ('can deleted pictures on instagram be recovered ?',
  'how do i delete a picture on instagram ?',
  0,
  'f965ab2d29d51008ab3872f47'),
 ('who is the best prime minister india has ever had ?',
  'who is the best prime minister of the india till now ?',
  1,
  '5aca073f4a977d55176f68572'),
 ('what qualifications do i need to get a job in cern ?',
  'what qualifications do i need to be able to work for cern ?',
  1,
  'a8abbd08d3401287da42acfb9'),
 ('in what situations is it okay to lie ?',
  'when is it okay to lie ?',
  1,
  '65dffaf8ac0ed7f3ebfcf0b02'),
 ('how do off campus placements take place ?',
  'how do i get off campus placement ?',
  0,
  '2c84e2535e3fda2a283c5dc14'),
 ('what is the difference between oracle 9i and 10i ?',
  'are oracle soa and oracle osb completely different , or one is one a part of the other one (this question comes from two archi

In [16]:
batched_para_data = para_train_data.collate_fn(para_train_data.dataset)

In [22]:
batched_para_data.keys()

dict_keys(['token_ids_1', 'token_type_ids_1', 'attention_mask_1', 'token_ids_2', 'token_type_ids_2', 'attention_mask_2', 'labels', 'sent_ids'])

In [25]:
batched_para_data['token_ids_1']

tensor([[  101,  2129,  2064,  ...,     0,     0,     0],
        [  101,  2064, 17159,  ...,     0,     0,     0],
        [  101,  2040,  2003,  ...,     0,     0,     0],
        ...,
        [  101,  2054,  2024,  ...,     0,     0,     0],
        [  101,  1999,  1038,  ...,     0,     0,     0],
        [  101,  2054,  2003,  ...,     0,     0,     0]])

In [24]:
batched_para_data['token_ids_2'].shape

torch.Size([141498, 286])

In [ ]:
class BertModel(BertPreTrainedModel):
  """
  the bert model returns the final embeddings for each token in a sentence
  it consists
  1. embedding (used in self.embed)
  2. a stack of n bert layers (used in self.encode)
  3. a linear transformation layer for [CLS] token (used in self.forward, as given)
  """
  def __init__(self, config):
    super().__init__(config)
    self.config = config

    # embedding
    self.word_embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
    self.pos_embedding = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.tk_type_embedding = nn.Embedding(config.type_vocab_size, config.hidden_size)
    self.embed_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.embed_dropout = nn.Dropout(config.hidden_dropout_prob)
    # position_ids (1, len position emb) is a constant, register to buffer
    position_ids = torch.arange(config.max_position_embeddings).unsqueeze(0)
    self.register_buffer('position_ids', position_ids)

    # bert encoder
    self.bert_layers = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])

    # for [CLS] token
    self.pooler_dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.pooler_af = nn.Tanh()

    self.init_weights()

  def embed(self, input_ids): #input_ids_1, input_ids_2
    
    input_shape = input_ids.size()
    seq_length = input_shape[1]

    # Get word embedding from self.word_embedding into input_embeds.
    inputs_embeds = self.word_embedding(input_ids)

    # Get position index and position embedding from self.pos_embedding into pos_embeds.
    pos_ids = self.position_ids[:, :seq_length] #subsets a list of positions 0:512 to 0:seq_length. 

    pos_embeds = self.pos_embedding(pos_ids)


    # Get token type ids, since we are not consider token type, just a placeholder.
    tk_type_ids = torch.zeros(input_shape, dtype=torch.long, device=input_ids.device)
    tk_type_embeds = self.tk_type_embedding(tk_type_ids)

    # Add three embeddings together; then apply embed_layer_norm and dropout and return.
    hidden_states = inputs_embeds+pos_embeds+tk_type_embeds
    hidden_states = self.embed_layer_norm(hidden_states)
    hidden_states = self.embed_dropout(hidden_states)

    return hidden_states


  def encode(self, hidden_states, attention_mask):
    """
    hidden_states: the output from the embedding layer [batch_size, seq_len, hidden_size]
    attention_mask: [batch_size, seq_len]
    """
    # get the extended attention mask for self attention
    # returns extended_attention_mask of [batch_size, 1, 1, seq_len]
    # non-padding tokens with 0 and padding tokens with a large negative number 
    extended_attention_mask: torch.Tensor = get_extended_attention_mask(attention_mask, self.dtype)

    # pass the hidden states through the encoder layers
    for i, layer_module in enumerate(self.bert_layers):
      # feed the encoding from the last bert_layer to the next
      hidden_states = layer_module(hidden_states, extended_attention_mask)

    return hidden_states

  def forward(self, input_ids, attention_mask):
    """
    input_ids: [batch_size, seq_len], seq_len is the max length of the batch
    attention_mask: same size as input_ids, 1 represents non-padding tokens, 0 represents padding tokens
    """
    # get the embedding for each input token
    embedding_output = self.embed(input_ids=input_ids)

    # feed to a transformer (a stack of BertLayers)
    sequence_output = self.encode(embedding_output, attention_mask=attention_mask)

    # get cls token hidden state
    first_tk = sequence_output[:, 0]
    first_tk = self.pooler_dense(first_tk)
    first_tk = self.pooler_af(first_tk)

    return {'last_hidden_state': sequence_output, 'pooler_output': first_tk}
